In [ ]:
# Colab 버전
!pip install opencv-python==4.9.0.80
!pip install numpy==1.26.4
!pip install mediapipe==0.10.11

!pip install CodingRider
!pip install keyboard

In [ ]:
# 사전 설정
# CodingRider 모듈 존재하는지 확인 후, 없을 경우 os모듈의 system을 이용해 설치
try:
    from CodingRider.drone import *
    from CodingRider.protocol import *
except ModuleNotFoundError:
    import os
    os.system("pip install CodingRider")
    from CodingRider.drone import *
    from CodingRider.protocol import *
except Exception as e:
    print(e)
    traceback.print_exc() # type: ignore , pylance 예외 처리

# keyboard 모듈 존재하는지 확인 후, 없을 경우 설치
try:
    import keyboard
except ModuleNotFoundError:
    import os
    os.system("pip install keyboard")
except Exception as e:
    print(e)
    traceback.print_exc() # type: ignore , pylance 예외 처리

In [ ]:
# 포트 확인 후 알맞은 포트 이름을 입력해야 정상적으로 작동
# 예제는 실험 환경이었던 COM4로 진행
# 장치 관리자(Device manager)의 포트(Port) 항목에서 연결된 기기 확인 가능
OPEN_PORT = 'COM4'

In [ ]:
import cv2
import mediapipe as mp
from time import sleep
import keyboard
from CodingRider.drone import *
from CodingRider.protocol import *

# is_takeoff 변수로 이륙했는지 확인합니다.
is_takeoff = False

drone = Drone()
drone.open('COM4')

capture = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as hands:

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            continue 
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:            
            for hand_landmarks in results.multi_hand_landmarks: 
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS) 
                finger = hand_landmarks.landmark[8] 
                # 이륙했다면 검지 손가락 좌표로 드론을 움직입니다.   
                if is_takeoff:                
                    # 롤과 피치 값을 정합니다.
                    roll = int(((finger.x * 200) - 100) * 0.5)
                    pitch = -int(((finger.y * 200) - 100) * 0.5)
                    drone.sendControl(roll, pitch, 0, 0)

                    # 롤과 피치 값을 나타냅니다.    
                    text = f'r : {roll}, p : {pitch}'
                    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 1)                
        cv2.imshow('MediaPipe Hand Detection', frame)
        if keyboard.is_pressed('enter'):   
            print('이륙')       
            sleep(2)
            drone.sendTakeOff()
            sleep(5)
            print('이륙 완료') 
            is_takeoff = True
        if keyboard.is_pressed('space'):
            print('착륙')
            drone.sendControlWhile(0, 0, 0, 0, 500) # 제자리에서 0.5초 멈춥니다.
            drone.sendLanding()
            sleep(3)
            print('착륙 완료')
            is_takeoff = False
        if keyboard.is_pressed('q'):
            print('정지')
            drone.sendControlWhile(0, 0, 0, 0, 500)
            drone.sendStop()
            sleep(3)
            is_takeoff = False
        if cv2.waitKey(1) == 27:
            break
        
capture.release()
cv2.destroyAllWindows()